In [1]:
%load_ext sql

In [2]:
%sql postgresql://appdev@data/appdev


'Connected: appdev@appdev'


# Assignment 7: Grouping and more joins
*Emmely Lundberg cph-el69*

This assignment focuses on the use of GROUP BY and different types of joins. Proceed stepwise and take everything very slowly to avoid getting very confused.


__At the bottom full query results are shown in the answers query results are limited to ten to not clutter the answers__

##### 1. Using a single join and a grouping, write a query that allows you to find the forename, surname, driverid and the total number of wins for each driver in the f1db schema.

In [136]:
%%sql SELECT * FROM information_schema.tables WHERE table_schema='f1db';
SELECT forename, surname, driverid, count(rank) as total_number_of_wins FROM drivers
JOIN results USING(driverid) WHERE results.rank = 1 GROUP BY driverid ORDER BY total_number_of_wins DESC;


13 rows affected.
31 rows affected.


forename,surname,driverid,total_number_of_wins
Kimi,Räikkönen,8,41
Lewis,Hamilton,1,38
Sebastian,Vettel,20,31
Fernando,Alonso,4,22
Michael,Schumacher,30,21
Nico,Rosberg,3,20
Mark,Webber,17,18
Felipe,Massa,13,15
Daniel,Ricciardo,817,9
Jenson,Button,18,8


##### 2. Write a query that joins the tables results, constructors and drivers to show the amount of times a driver has driven a car from a constructor. In other words how many times one driver (for instance Schumacher) has driven a constructor (for instance Mercedes). Your table should have three fields: drivers.driverref, constructors.name and count(*) (showing the number of times the driver has driven the constructor).
*Hint: You can group on more than one value*


__Note I have limited the query to 10 results. At the bottom of the page the full results of the query are shown.__

In [137]:
%%sql SELECT driverref, name, count(*) from drivers 
JOIN results USING(driverid)
JOIN constructors USING(constructorid) 
GROUP BY driverref, name ORDER BY count DESC LIMIT 10;


10 rows affected.


driverref,name,count
michael_schumacher,Ferrari,181
coulthard,McLaren,150
massa,Ferrari,140
button,McLaren,137
rosberg,Mercedes,136
hakkinen,McLaren,133
laffite,Ligier,132
webber,Red Bull,129
raikkonen,Ferrari,126
vettel,Red Bull,113


##### 3. Now we have found the most driven vehicles even for those who didn't finish the race. Extend the query from 2 by removing all tuples from the result where the status is not "Finished".

__Note I have limited the query to 10 results. At the bottom of the page the full results of the query are shown.__

In [138]:
%%sql SELECT  driverref, name, count(*) from drivers 
JOIN results USING(driverid)
JOIN constructors USING(constructorid) 
WHERE statusid = 1 
GROUP BY driverref, name ORDER BY name, driverref DESC LIMIT 10;

10 rows affected.


driverref,name,count
stuck,ATS,1
mass,ATS,1
reg_parnell,Alfa Romeo,1
mario_andretti,Alfa Romeo,2
giacomelli,Alfa Romeo,4
farina,Alfa Romeo,7
fangio,Alfa Romeo,8
fagioli,Alfa Romeo,6
cesaris,Alfa Romeo,3
baldi,Alfa Romeo,1


##### 4. Create a third (and new) query that finds the amount of milliseconds spent in pitstop (see the pitstops relation) by each unique combinations of driverid and raceid.
*Hint: use the sum aggregate function to find the total amount of milliseconds*

*Hint: i recommend making this into a view*


__Note I have limited the query to 10 results. At the bottom of the page the full results of the query are shown.__

I create the view that I can reuse in the following queries.

In [139]:
%sql SELECT * FROM pitstops;
%sql SELECT pitstops.driverid, pitstops.raceid, sum(milliseconds) as milliseconds FROM pitstops GROUP BY pitstops.driverid, pitstops.raceid;
%sql CREATE VIEW milliseconds_spend AS SELECT pitstops.driverid, pitstops.raceid, sum(milliseconds) as milliseconds FROM pitstops GROUP BY pitstops.driverid, pitstops.raceid;

%sql SELECT * FROM milliseconds_spend LIMIT 10;


6090 rows affected.
2699 rows affected.
(psycopg2.ProgrammingError) relation "milliseconds_spend" already exists
 [SQL: 'CREATE VIEW milliseconds_spend AS SELECT pitstops.driverid, pitstops.raceid, sum(milliseconds) as milliseconds FROM pitstops GROUP BY pitstops.driverid, pitstops.raceid;']
10 rows affected.


driverid,raceid,milliseconds
817,967,3572090
18,967,3569001
830,967,3563738
13,967,3554774
826,967,3546767
825,967,3542704
822,967,3542270
821,967,3538573
20,967,3524541
836,967,3518952


##### 5.Time to put everything together. Use the query for 4 as a join subquery in query 3 (see slides on "a query within a query") to find the total amount of pitstop time, for each result in query 3


__Note I have limited the query to 10 results. At the bottom of the page the full results of the query are shown.__

In [140]:
%%sql SELECT drivers.driverref, constructors.name, count(constructors.name) FROM drivers
JOIN results USING (driverid) 
JOIN constructors USING (constructorid)  
INNER JOIN 
(  
    SELECT * FROM milliseconds_spend
) p 
ON p.driverid=drivers.driverid 
WHERE results.statusid=1
GROUP BY drivers.driverref, constructors.name ORDER BY constructors.name,drivers.driverref DESC LIMIT 10; 

10 rows affected.


driverref,name,count
rosa,Arrows,34
button,BAR,2997
vettel,BMW Sauber,125
heidfeld,BMW Sauber,470
michael_schumacher,Benetton,1672
button,Benetton,222
button,Brawn,1776
barrichello,Brawn,288
pic,Caterham,74
petrov,Caterham,74


##### 6. As a last thing we want to find the driver with the least pistop time. However we want to control for the amount of races (see explanation below), so we need to include one last column: the average pitstop time per race. Your table should be sorted based on the ascending values of that average pitstop time.
##### Explanation: If we just count the pitstop time, we only get the total time he spend in a workshop, no matter how many races he participated in. If a driver participated in 100 races, he would have more pitstop time than a driver participating in one race!
*Hint: Use the sum function to accumulate the total pistop time and divide that with the amount of races the driver has participated in*

In [141]:
%%sql SELECT drivers.driverref, constructors.name, count(*), sum(p.milliseconds) as pitstop_time_total, ROUND(sum(p.milliseconds)/count(results.raceid), 2) as average_pitstop_time_per_race  
FROM drivers
JOIN results USING (driverid) 
JOIN constructors USING (constructorid)  
INNER JOIN 
(  
    SELECT * FROM milliseconds_spend
) p 
ON p.driverid=drivers.driverid 
WHERE results.statusid=1
GROUP BY drivers.driverref, constructors.name
ORDER BY average_pitstop_time_per_race ASC LIMIT 10; 


10 rows affected.


driverref,name,count,pitstop_time_total,average_pitstop_time_per_race
glock,Marussia,72,3561592,49466.56
glock,Toyota,792,39177512,49466.56
pic,Marussia,74,3860832,52173.41
pic,Caterham,74,3860832,52173.41
resta,Force India,1972,103995834,52736.22
michael_schumacher,Benetton,1672,89264912,53388.11
michael_schumacher,Mercedes,1292,68977432,53388.11
michael_schumacher,Ferrari,5358,286053468,53388.11
rosa,HRT,17,934171,54951.24
rosa,McLaren,102,5605026,54951.24


### Full results not using Views

2.

In [142]:
%%sql SELECT driverref, name, count(*) from drivers 
JOIN results USING(driverid)
JOIN constructors USING(constructorid) 
GROUP BY driverref, name ORDER BY count DESC;


2093 rows affected.


driverref,name,count
michael_schumacher,Ferrari,181
coulthard,McLaren,150
massa,Ferrari,140
button,McLaren,137
rosberg,Mercedes,136
hakkinen,McLaren,133
laffite,Ligier,132
webber,Red Bull,129
raikkonen,Ferrari,126
vettel,Red Bull,113


3.

In [143]:
%%sql SELECT  driverref, name, count(*) from drivers 
JOIN results USING(driverid)
JOIN constructors USING(constructorid) 
WHERE statusid = 1 
GROUP BY driverref, name ORDER BY name, driverref DESC;

658 rows affected.


driverref,name,count
stuck,ATS,1
mass,ATS,1
reg_parnell,Alfa Romeo,1
mario_andretti,Alfa Romeo,2
giacomelli,Alfa Romeo,4
farina,Alfa Romeo,7
fangio,Alfa Romeo,8
fagioli,Alfa Romeo,6
cesaris,Alfa Romeo,3
baldi,Alfa Romeo,1


4.

In [144]:
%sql SELECT driverid, raceid, sum(milliseconds) as milliseconds FROM pitstops GROUP BY driverid, raceid ORDER BY milliseconds DESC;


2699 rows affected.


driverid,raceid,milliseconds
817,967,3572090
18,967,3569001
830,967,3563738
13,967,3554774
826,967,3546767
825,967,3542704
822,967,3542270
821,967,3538573
20,967,3524541
836,967,3518952


5.

In [145]:
%%sql SELECT drivers.driverref, constructors.name, count(constructors.name) from drivers
JOIN results USING (driverid) 
JOIN constructors USING (constructorid)  
INNER JOIN 
(  
    SELECT pitstops.driverid, pitstops.raceid, sum(milliseconds) as milliseconds 
    FROM pitstops GROUP BY pitstops.driverid, pitstops.raceid
) p 
ON p.driverid=drivers.driverid 
WHERE results.statusid=1
GROUP BY drivers.driverref, constructors.name ORDER BY constructors.name,drivers.driverref DESC; 

118 rows affected.


driverref,name,count
rosa,Arrows,34
button,BAR,2997
vettel,BMW Sauber,125
heidfeld,BMW Sauber,470
michael_schumacher,Benetton,1672
button,Benetton,222
button,Brawn,1776
barrichello,Brawn,288
pic,Caterham,74
petrov,Caterham,74


6.

In [146]:
%%sql SELECT drivers.driverref, constructors.name, count(*), sum(p.milliseconds) as pitstop_time_total, ROUND(sum(p.milliseconds)/count(results.raceid), 2) as average_pitstop_time_per_race  
FROM drivers
JOIN results USING (driverid) 
JOIN constructors USING (constructorid)  
INNER JOIN 
(  
    SELECT pitstops.driverid, pitstops.raceid, sum(milliseconds) as milliseconds 
    FROM pitstops GROUP BY pitstops.driverid, pitstops.raceid
) p 
ON p.driverid=drivers.driverid 
WHERE results.statusid=1
GROUP BY drivers.driverref, constructors.name
ORDER BY average_pitstop_time_per_race ASC; 


118 rows affected.


driverref,name,count,pitstop_time_total,average_pitstop_time_per_race
glock,Marussia,72,3561592,49466.56
glock,Toyota,792,39177512,49466.56
pic,Caterham,74,3860832,52173.41
pic,Marussia,74,3860832,52173.41
resta,Force India,1972,103995834,52736.22
michael_schumacher,Mercedes,1292,68977432,53388.11
michael_schumacher,Ferrari,5358,286053468,53388.11
michael_schumacher,Benetton,1672,89264912,53388.11
rosa,Sauber,85,4670855,54951.24
rosa,HRT,17,934171,54951.24
